# DATA SKEWNESS SPARK_PARTITION_ID

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/27 18:45:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [8]:
schema = StructType([
    StructField("Year",IntegerType(),False),
    StructField("First Name",StringType(),True),
    StructField("County",StringType(),True),
    StructField("Sex",StringType(),True),
    StructField("Count",IntegerType(),True),
    
])

In [9]:
df_names = spark.read.format("csv").schema(schema).option("header","true").load("../datasets/baby_names.csv")

In [10]:
df_names.show()

+----+----------+-----------+---+-----+
|Year|First Name|     County|Sex|Count|
+----+----------+-----------+---+-----+
|2007|      ZOEY|      KINGS|  F|   11|
|2007|      ZOEY|    SUFFOLK|  F|    6|
|2007|      ZOEY|     MONROE|  F|    6|
|2007|      ZOEY|       ERIE|  F|    9|
|2007|       ZOE|     ULSTER|  F|    5|
|2007|       ZOE|WESTCHESTER|  F|   24|
|2007|       ZOE|      BRONX|  F|   13|
|2007|       ZOE|   NEW YORK|  F|   55|
|2007|       ZOE|     NASSAU|  F|   15|
|2007|       ZOE|       ERIE|  F|    6|
|2007|       ZOE|    SUFFOLK|  F|   14|
|2007|       ZOE|      KINGS|  F|   34|
|2007|       ZOE|     MONROE|  F|    9|
|2007|       ZOE|     QUEENS|  F|   26|
|2007|       ZOE|     ALBANY|  F|    5|
|2007|     ZISSY|   ROCKLAND|  F|    5|
|2007|     ZISSY|      KINGS|  F|   27|
|2007|      ZION|      KINGS|  M|   15|
|2007|      ZION|      BRONX|  M|   14|
|2007|       ZEV|   ROCKLAND|  M|    6|
+----+----------+-----------+---+-----+
only showing top 20 rows



In [11]:
df_names.rdd.getNumPartitions()

1

## CHECK PARTITION ID

In [12]:
from pyspark.sql.functions import spark_partition_id

In [14]:
df_names.withColumn("partition_id",spark_partition_id()).show()

+----+----------+-----------+---+-----+------------+
|Year|First Name|     County|Sex|Count|partition_id|
+----+----------+-----------+---+-----+------------+
|2007|      ZOEY|      KINGS|  F|   11|           0|
|2007|      ZOEY|    SUFFOLK|  F|    6|           0|
|2007|      ZOEY|     MONROE|  F|    6|           0|
|2007|      ZOEY|       ERIE|  F|    9|           0|
|2007|       ZOE|     ULSTER|  F|    5|           0|
|2007|       ZOE|WESTCHESTER|  F|   24|           0|
|2007|       ZOE|      BRONX|  F|   13|           0|
|2007|       ZOE|   NEW YORK|  F|   55|           0|
|2007|       ZOE|     NASSAU|  F|   15|           0|
|2007|       ZOE|       ERIE|  F|    6|           0|
|2007|       ZOE|    SUFFOLK|  F|   14|           0|
|2007|       ZOE|      KINGS|  F|   34|           0|
|2007|       ZOE|     MONROE|  F|    9|           0|
|2007|       ZOE|     QUEENS|  F|   26|           0|
|2007|       ZOE|     ALBANY|  F|    5|           0|
|2007|     ZISSY|   ROCKLAND|  F|    5|       

In [15]:
df_names_re = df_names.repartition(20).withColumn("partition_id",spark_partition_id())

df_names_re.show(100)

+----+-----------+-----------+---+-----+------------+
|Year| First Name|     County|Sex|Count|partition_id|
+----+-----------+-----------+---+-----+------------+
|2008|     BRENDA|   ROCKLAND|  F|    5|           0|
|2011|     ALEXIS|      KINGS|  M|   14|           0|
|2014|   ISABELLA|     Oswego|  F|    7|           0|
|2010|    AALIYAH|   DUTCHESS|  F|    7|           0|
|2014|     MORGAN|     Monroe|  F|   10|           0|
|2009|      DYLAN|    SUFFOLK|  M|   87|           0|
|2009|     SAWYER|       ERIE|  M|    5|           0|
|2009|     MARIAM|      BRONX|  F|   17|           0|
|2014|     GEORGE|       Erie|  M|    5|           0|
|2008|     GIANNA|       ERIE|  F|   35|           0|
|2013|       ADAM|   DUTCHESS|  M|    5|           0|
|2014|   KINGSTON|      Kings|  M|    9|           0|
|2013|       NORA|     MONROE|  F|    9|           0|
|2008|      GRACE|     ONEIDA|  F|    9|           0|
|2012|  ALEXANDER|   SARATOGA|  M|   15|           0|
|2012|    MADELYN|     MONRO

In [16]:
df_names_re.rdd.getNumPartitions()

20

## Show Records Per Partition

In [18]:
from pyspark.sql.functions import asc, desc

In [19]:
df_names_re.groupBy("partition_id").count().orderBy(desc("count")).show()

+------------+-----+
|partition_id|count|
+------------+-----+
|           0| 2613|
|           9| 2613|
|          10| 2613|
|          11| 2613|
|          12| 2613|
|          13| 2613|
|          14| 2613|
|          15| 2613|
|          16| 2613|
|          17| 2613|
|          18| 2613|
|          19| 2613|
|           1| 2612|
|           2| 2612|
|           3| 2612|
|           4| 2612|
|           5| 2612|
|           6| 2612|
|           7| 2612|
|           8| 2612|
+------------+-----+

